In [1]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# data
#
from techminer import *


# Records(directory="../data/processed/debug").import_records(source="../data/tmp/debug-scopus.csv", filetype="scopus")
Records(directory="../data/processed/debug").process_records()

# wos_df = load_file(filepath="techminer/tests/data/savedrecs.txt", filetype="wos", datastorepath="./datastore")
# dim_df = load_file(filepath="techminer/tests/data/dimensions.csv", filetype="dimensions", datastorepath="./datastore")

2021-10-30 20:49:40 - INFO - Processing records ...
2021-10-30 20:49:40 - INFO - Creating institutions thesaurus ...
2021-10-30 20:49:40 - INFO - Affiliations without country detected - check file ../data/processed/debug/ignored_affiliations.txt
2021-10-30 20:49:40 - INFO - Affiliations without country detected - check file ../data/processed/debug/ignored_affiliations.txt
2021-10-30 20:49:40 - INFO - Thesaurus file '../data/processed/debug/institutions.txt' created.
2021-10-30 20:49:40 - INFO - Creating keywords thesaurus ...
2021-10-30 20:49:42 - INFO - Thesaurus file '../data/processed/debug/keywords.txt' created.
2021-10-30 20:49:42 - INFO - Searching local references using DOI ...
2021-10-30 20:49:48 - INFO - Searching local references using document titles ...
2021-10-30 20:49:55 - INFO - Consolidating local references ...
2021-10-30 20:49:55 - INFO - Computing local citations ...
2021-10-30 20:49:55 - INFO - Computing Bradford Law Zones ...
2021-10-30 20:49:55 - INFO - The file '

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.data import *

# coverage(directory_or_records="../data/processed/debug")
describe_records(directory_or_records="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# features
#
from techminer.features import *

# extract_coutries(directory_or_records="../data/processed/debug", column='affiliations')






In [ ]:
#
# query modules
#
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.query import * 
from techminer.visualization import worldmap



# term_analysis(directory_or_records="../data/processed/debug", column='authors')
### count_terms_by_column(directory_or_records="../data/processed/debug", sep="; ", column='authors')
# time_analysis(directory_or_records="../data/processed/debug")
### count_local_citations_by_year(directory_or_records="../data/processed/debug")
### count_global_citations_by_year(directory_or_records="../data/processed/debug")
### count_documents_by_year(directory_or_records="../data/processed/debug")
### mean_global_citations_by_year(directory_or_records="../data/processed/debug")
# most_cited_documents(directory_or_records="../data/processed/debug")
# compute_impact_index(directory_or_records="../data/processed/debug", column='authors')




# ---
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column='countries'))



In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import Records

Records(directory=)

# core_sources(directory_or_records="../data/processed/debug")
# core_authors(directory_or_records="../data/processed/debug")
# lotka(core_authors(directory_or_records="../data/processed/debug"), colormap='Blues', figsize=(6,3))


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from techminer.visualization import bradford

bradford(directory_or_records="../data/processed/debug")

In [ ]:
import pandas as pd

from techminer.data.records import load_records

# sorted(load_records(directory="../data/processed/debug").columns)
load_records(directory="../data/processed/debug")

In [ ]:
from src.features.apply_keywords_thesaurus import apply_keywords_thesaurus
from src.features.apply_institutions_thesaurus import apply_institutions_thesaurus

# Ok!
apply_keywords_thesaurus(datastoredir="../data/processed/debug")
apply_institutions_thesaurus(datastoredir="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.analysis import Bradford

bradford = Bradford(datastoredir="../data/processed/debug")

bradford.plot(max_items=8, cmap='Blues', figsize=(8, 5), fontsize=9)

In [ ]:
bradford.sources_

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from src.analysis import TopDocuments

TopDocuments(datastoredir="../data/processed/debug").detailed_

In [ ]:
from techminer.analysis.top_documents import TopDocuments

TopDocuments(datastorepath="./datastore").print()

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").plot(figsize=(8, 4))

In [ ]:
from techminer.analysis.core_sources import CoreSources

CoreSources(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.coverage import Coverage

Coverage(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.worldmap import WorldMap

WorldMap(datastorepath="./datastore").plot(figsize=(9, 5), colormap='Blues')

In [2]:
import pandas as pd

pd.read_csv("../data/processed/debug/records.csv").head()

FileNotFoundError: [Errno 2] No such file or directory: './datastore/records.csv'